In [142]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [13]:
from google.colab import drive
drive.mount('/content/mnt')

Drive already mounted at /content/mnt; to attempt to forcibly remount, call drive.mount("/content/mnt", force_remount=True).


In [14]:
data_folder = '/content/mnt/MyDrive/Data/서울시구별CCTV'

In [15]:
!ls -l $data_folder

total 6
-rw------- 1 root root 1793 Sep  7 05:23 '서울시CCTV설치운영현황(자치구)_년도별_210731기준.csv'
-rw------- 1 root root 3179 Sep  7 05:25 '서울시주민등록인구(구별)통계.tsv'


### 서울 CCTV 현황

In [79]:
df = pd.read_csv(f'{data_folder}/서울시CCTV설치운영현황(자치구)_년도별_210731기준.csv', encoding='euc-kr')
print(df.shape)
df.head(2)

(27, 13)


,"※ 2021.7.31. 현황을 기준으로 각 연도별 설치된 CCTV 수량. 교체(저화질교체, 성능개선)는 최초설치연도가 아닌 교체년도 수량에 포함",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
1,계,"77,032","7,667","2,200","3,491","4,439","6,582","8,129","9,947","9,876","11,961","11,132","1,608"


In [80]:
df.rename(columns= df.iloc[0], inplace=True)
df = df.drop([0])
df = df.drop([1])
del df['2021년']
df.reset_index(drop=True, inplace=True)
df.head()

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년
0,종로구,"1,772",813,0,0,210,150,1,261,85,9,200
1,중 구,"2,333",16,114,87,77,236,240,372,386,155,361
2,용산구,"2,383",34,71,234,125,221,298,351,125,307,617
3,성동구,"3,602",448,125,212,105,339,310,874,390,262,461
4,광진구,"2,588",35,57,100,187,98,52,675,465,712,175


In [120]:
# 구분, 총계만 출력 
cctv = df[['구분', '총계']]
cctv.rename(columns = {'구분':'자치구', '총계':'CCTV수'}, inplace=True)
cctv.head()

,자치구,CCTV수
0,종로구,"1,772"
1,중 구,"2,333"
2,용산구,"2,383"
3,성동구,"3,602"
4,광진구,"2,588"


##  인구 데이터

In [113]:
pop = pd.read_csv(f'{data_folder}/서울시주민등록인구(구별)통계.tsv', sep='\t')
pop.head()

,기간,자치구,세대,인구,인구.1,인구.2,인구.3,인구.4,인구.5,인구.6,인구.7,인구.8,세대당인구,65세이상고령자
0,기간,자치구,세대,합계,합계,합계,한국인,한국인,한국인,등록외국인,등록외국인,등록외국인,세대당인구,65세이상고령자
1,기간,자치구,세대,계,남자,여자,계,남자,여자,계,남자,여자,세대당인구,65세이상고령자
2,2021.2/4,합계,"4,405,638","9,795,426","4,756,178","5,039,248","9,565,990","4,649,442","4,916,548","229,436","106,736","122,700",2.17,"1,576,971"
3,2021.2/4,종로구,"73,525","155,106","75,009","80,097","146,029","70,985","75,044","9,077","4,024","5,053",1.99,"27,605"
4,2021.2/4,중구,"62,897","132,259","64,526","67,733","123,016","60,019","62,997","9,243","4,507","4,736",1.96,"23,980"


In [114]:
pop = pop[['자치구','세대','인구','인구.3','인구.6','세대당인구','65세이상고령자']]
pop.head()

,자치구,세대,인구,인구.3,인구.6,세대당인구,65세이상고령자
0,자치구,세대,합계,한국인,등록외국인,세대당인구,65세이상고령자
1,자치구,세대,계,계,계,세대당인구,65세이상고령자
2,합계,"4,405,638","9,795,426","9,565,990","229,436",2.17,"1,576,971"
3,종로구,"73,525","155,106","146,029","9,077",1.99,"27,605"
4,중구,"62,897","132,259","123,016","9,243",1.96,"23,980"


In [115]:
pop.rename(columns= pop.iloc[0], inplace=True)
pop= pop.drop([0])
pop= pop.drop([1])
pop= pop.drop([2])
pop.reset_index(drop=True, inplace=True)
pop.head()

,자치구,세대,합계,한국인,등록외국인,세대당인구,65세이상고령자
0,종로구,"73,525","155,106","146,029","9,077",1.99,"27,605"
1,중구,"62,897","132,259","123,016","9,243",1.96,"23,980"
2,용산구,"111,650","240,665","226,378","14,287",2.03,"38,884"
3,성동구,"134,542","295,767","289,162","6,605",2.15,"45,591"
4,광진구,"166,957","355,306","342,481","12,825",2.05,"50,815"


In [146]:
# 자치구, 세대, 합계만 출력 
pop = pop[['자치구','세대','합계']]
pop.head()

,자치구,세대,합계
0,종로구,"73,525","155,106"
1,중구,"62,897","132,259"
2,용산구,"111,650","240,665"
3,성동구,"134,542","295,767"
4,광진구,"166,957","355,306"


# 각 구의 세대별 CCTV 수
- [세계 최고 ‘감시도시’는 베이징·상하이·런던…서울은 44위](https://www.hani.co.kr/arti/international/international_general/955343.html)

In [133]:
f_cctv= pd.merge(family,cctv)
f_cctv.rename(columns = {'자치구':'ID','세대':'세대','CCTV수':'CCTV수'}, inplace=True)
f_cctv.head()

,ID,세대,CCTV수
0,종로구,"73,525","1,772"
1,용산구,"111,650","2,383"
2,성동구,"134,542","3,602"
3,광진구,"166,957","2,588"
4,동대문구,"166,372","2,497"


In [134]:
f_cctv['CCTV수'] = f_cctv.CCTV수.apply(lambda x: int(x.replace(',','')))
f_cctv['세대'] = f_cctv.세대.apply(lambda x: int(x.replace(',','')))
f_cctv.head()

,ID,세대,CCTV수
0,종로구,73525,1772
1,용산구,111650,2383
2,성동구,134542,3602
3,광진구,166957,2588
4,동대문구,166372,2497


In [138]:
f_cctv['세대별CCTV'] = f_cctv.CCTV수 / f_cctv.세대
f_cctv.head()

,ID,세대,CCTV수,세대별CCTV
0,종로구,73525,1772,0.024101
1,용산구,111650,2383,0.021343
2,성동구,134542,3602,0.026772
3,광진구,166957,2588,0.015501
4,동대문구,166372,2497,0.015009


In [147]:
f_cctv.to_csv(f'{data_folder}/세대별_CCTV.csv', index=False)

### 인구별 CCTV 

In [180]:
p_cctv= pd.merge(pop,cctv)
p_cctv.rename(columns = {'자치구':'자치구','세대':'세대','합계':'인구','CCTV수':'CCTV수'}, inplace=True)
p_cctv.head()

,자치구,세대,인구,CCTV수
0,종로구,"73,525","155,106","1,772"
1,용산구,"111,650","240,665","2,383"
2,성동구,"134,542","295,767","3,602"
3,광진구,"166,957","355,306","2,588"
4,동대문구,"166,372","352,259","2,497"


In [181]:
p_cctv['CCTV수'] = p_cctv.CCTV수.apply(lambda x: int(x.replace(',','')))
p_cctv['세대'] = p_cctv.세대.apply(lambda x: int(x.replace(',','')))
p_cctv['인구'] = p_cctv.인구.apply(lambda x: int(x.replace(',','')))
p_cctv.head()

,자치구,세대,인구,CCTV수
0,종로구,73525,155106,1772
1,용산구,111650,240665,2383
2,성동구,134542,295767,3602
3,광진구,166957,355306,2588
4,동대문구,166372,352259,2497


In [182]:
p_cctv['인구(천명)'] = p_cctv.인구/1000
del p_cctv['인구']
p_cctv.head()

,자치구,세대,CCTV수,인구(천명)
0,종로구,73525,1772,155.106
1,용산구,111650,2383,240.665
2,성동구,134542,3602,295.767
3,광진구,166957,2588,355.306
4,동대문구,166372,2497,352.259


In [183]:
# 인구 100명 당 CCTV 수
p_cctv['인구별CCTV'] = p_cctv.CCTV수 / p_cctv['인구(천명)']
p_cctv.head()

,자치구,세대,CCTV수,인구(천명),인구별CCTV
0,종로구,73525,1772,155.106,11.424445
1,용산구,111650,2383,240.665,9.901731
2,성동구,134542,3602,295.767,12.178505
3,광진구,166957,2588,355.306,7.283862
4,동대문구,166372,2497,352.259,7.088534


In [184]:
# 세대별 CCTV 수 
p_cctv['세대(백)'] = p_cctv.세대 /100
p_cctv['세대별CCTV'] = p_cctv.CCTV수/p_cctv['세대(백)']
p_cctv.head()

,자치구,세대,CCTV수,인구(천명),인구별CCTV,세대(백),세대별CCTV
0,종로구,73525,1772,155.106,11.424445,735.25,2.410065
1,용산구,111650,2383,240.665,9.901731,1116.50,2.134348
2,성동구,134542,3602,295.767,12.178505,1345.42,2.677231
3,광진구,166957,2588,355.306,7.283862,1669.57,1.550100
4,동대문구,166372,2497,352.259,7.088534,1663.72,1.500854


In [189]:
p_cctv.to_csv(f'{data_folder}/서울시구별_CCTV.csv', index=False)